In [0]:
%sql

-- SELECT * FROM bronze.music_data.albums
-- On Air - Live At The BBC (Vol.2)

-- UPDATE bronze.music_data.albums SET album_title="On Air - Live At The BBC (Vol.2)" WHERE album_id=67272022

-- SELECT * FROM bronze.music_data.albums WHERE album_id=67272022
-- SELECT * FROM silver.music_data.albums WHERE albumId=67272022
-- SELECT * from table_changes("bronze.music_data.albums", 0, 2)

-- select 

-- select * from table_changes("bronze.music_data.albums", 0, 3)

-- -- SELECT COUNT(*) FROM silver.music_data.albums
-- SELECT COUNT(*) FROM bronze.music_data.albums
-- SELECT * FROM silver.music_data.albums WHERE albumId=132
-- DROP TABLE silver.music_data.albums;
-- DROP TABLE silver.music_data.tracks;
-- DROP TABLE silver.music_data.albums_genres;
-- DROP TABLE silver.music_data.genres;
-- DROP TABLE silver.music_data.artists;
-- DROP TABLE bronze.music_data.albums;
-- DROP TABLE bronze.music_data.tracks;
-- DROP TABLE bronze.music_data.albums_genres;
-- DROP TABLE bronze.music_data.genres;
-- DROP TABLE bronze.music_data.artists;


In [0]:
import sys
# sys.path.append(f'/Workspace/Users/{dbutils.widgets.get("account")}/music_data_lake/src/lib')
sys.path.append(f'/Workspace/Users/mydatabrickstestacc@gmail.com/music_data_lake/src/lib')
# sys.path.insert(0, '../lib')

import utils
import ingestors

config = {
    "catalog": "silver",
    "schema": "music_data",
    "tablename": "albums",
    "primary_key": "albumId",
    "id_field_old": "album_id",
    "timestamp_field": "_commit_timestamp"
}

In [0]:
config = {
    "catalog": dbutils.widgets.get("catalog"),
    "schema": dbutils.widgets.get("schema"),
    "tablename": dbutils.widgets.get("tablename"),
    "primary_key": dbutils.widgets.get("primary_key"),
    "timestamp_field": dbutils.widgets.get("timestamp_field"),
    "id_field_old": dbutils.widgets.get("id_field_old")
}

In [0]:
remove_checkpoint = False

if not utils.table_exists(spark, config["catalog"], config["schema"], config["tablename"]):

    print("Criando a tabela", config["tablename"])
    path = f"{config['tablename']}.sql"
    with open(path, "r") as open_file:
        query = open_file.read()
    (spark.sql(query)
          .write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema", "true")
          .saveAsTable(f"{config['catalog']}.{config['schema']}.{config['tablename']}"))
    
    remove_checkpoint = True

else:
    print("tabela ja existente")

In [0]:
print("Iniciando CDF...")

ingest = ingestors.SilverIngestor(spark=spark,
                               config=config)

In [0]:
if remove_checkpoint:
    dbutils.fs.rm(ingest.checkpoint_location, True)

In [0]:
stream = ingest.run()